In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [2]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-hf"

# The instruction dataset to use
dataset_name = "nadiamaqbool81/python_code_instructions_510_alpaca"

# Fine-tuned model name
new_model = "llama-2-7b-int4-python-code-510"

hf_model_repo="nadiamaqbool81/"+new_model

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [3]:
from huggingface_hub import notebook_login
# Log in to HF Hub
notebook_login()

In [4]:
import torch
from transformers import T5ForConditionalGeneration, AutoTokenizer, RobertaTokenizer,AutoModelForCausalLM

# Get the tokenizer
tokenizer = AutoTokenizer.from_pretrained(hf_model_repo)
# Load the model
model = AutoModelForCausalLM.from_pretrained(hf_model_repo, load_in_4bit=True, torch_dtype=torch.float16,
                                             device_map=device_map)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from transformers import AutoTokenizer, T5ForConditionalGeneration, AutoModelForCausalLM

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

In [6]:
args = training_arguments

In [7]:
import random
def run_predict(args, text):

    prompt = text
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=300)
    result = pipe(f"<s>[INST] {prompt} [/INST]")
    arr = result[0]['generated_text'].split('[/INST]')
    return arr[1]


In [8]:
def calc_bleu(decoded_code, code):
    # Calculate the BLEU score.
    if len(decoded_code):
        bleu_score = nltk.translate.bleu_score.sentence_bleu(decoded_code, code)
    else:
        bleu_score = 0

    return bleu_score

In [10]:
import json
import nltk
import pandas as pd
import re

def predict_from_datasets(args):
    # load using hf datasets
    with open('/kaggle/input/pythontestdata/pythonTestData.json') as f:
       dataset = json.load(f)
    average_bleu_score = []
    bleu_scores = []
    for row in dataset:
      # Extract the generated text and the reference text.
      text = row["instruction"]
      code = row["code"]
      # run-predict on text
      decoded_code = run_predict(args, text)
      generated_text = decoded_code.split()
      reference_text = code.split()
      print("#" * 25); print("QUERY: ", text);
      print()
      print('#' * 25); print("ORIGINAL: "); print("\n", code);
      print()
      print('#' * 25); print("GENERATED: "); print("\n", decoded_code);
      print()
      # Calculate the BLEU score.
      bleu_score=calc_bleu(generated_text, reference_text)
      print("bleu_score:  " ,bleu_score)
      bleu_scores.append(bleu_score)
      # Average the BLEU scores for all rows.
      average_bleu_score = sum(bleu_scores) / len(bleu_scores)
      print("average_bleu_score:  " ,average_bleu_score)
      output   = re.sub(r"[\n\t]*", "", decoded_code)
      try:
          with open('/kaggle/working/llamaPredictions.txt', 'a') as f:
            f.write(output)
            f.write("\n")
      except FileNotFoundError:
          print("The 'docs' directory does not exist")

    # Print the average BLEU score.
    print("average_bleu_score:  " ,average_bleu_score)
    return average_bleu_score

In [11]:
score = predict_from_datasets(args)
print("BLEU SCORE FOR FINE-TUNED LLAMA2 MODEL ON TEST DATA AGAIN: " , score)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


#########################
QUERY:  Write a python function to find minimum sum of factors of a given number.

#########################
ORIGINAL: 

 def find_Min_Sum(num): 
    sum = 0
    i = 2
    while(i * i <= num): 
        while(num % i == 0): 
            sum += i 
            num /= i 
        i += 1
    sum += num 
    return sum

#########################
GENERATED: 

  def find_min_sum_factors(n):
  factors = set(range(1, n + 1))
  for i in range(1, n + 1):
    for j in range(1, i + 1):
      if i % j == 0:
        factors.add(j)
  return min(factors)  
  
find_min_sum_factors(1000000)  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  


bleu_score:   0.6838911999336902
average_bleu_score:   0.6838911999336902


/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


#########################
QUERY:  Write a function to count the element frequency in the mixed nested tuple.

#########################
ORIGINAL: 

 def flatten(test_tuple): 
	for tup in test_tuple: 
		if isinstance(tup, tuple): 
			yield from flatten(tup) 
		else: 
			yield tup 
def count_element_freq(test_tuple):
  res = {}
  for ele in flatten(test_tuple):
    if ele not in res:
      res[ele] = 0
    res[ele] += 1
  return (res) 

#########################
GENERATED: 

  def count_element_frequency(tup):
  return tuple(count(element) for element in tup)
  
def count(element):
  return element in element_count
  
element_count = {}
  
def count_element_frequency_2(tup):
  return tuple(count(element) for element in tup)
  
def count_element_frequency_3(tup):
  return tuple(count(element) for element in tup)
  
def count_element_frequency_4(tup):
  return tuple(count(element) for element in tup)
  
def count_element_frequency_5(tup):
  return tuple(count(element) for element in tup)
 

In [12]:
score = predict_from_datasets(args)
print("BLEU SCORE FOR FINE-TUNED LLAMA2 MODEL ON TEST DATA: " , score)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


#########################
QUERY:  Write a python function to find minimum sum of factors of a given number.

#########################
ORIGINAL: 

 def find_Min_Sum(num): 
    sum = 0
    i = 2
    while(i * i <= num): 
        while(num % i == 0): 
            sum += i 
            num /= i 
        i += 1
    sum += num 
    return sum

#########################
GENERATED: 

  def find_min_sum_factors(n):
  factors = set(range(1, n + 1))
  for i in range(1, n + 1):
    for j in range(1, i + 1):
      if i % j == 0:
        factors.add(j)
  return min(factors)  
  
find_min_sum_factors(1000000)  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  

bleu_score:   0.683891199933

/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


#########################
QUERY:  Write a function to count the element frequency in the mixed nested tuple.

#########################
ORIGINAL: 

 def flatten(test_tuple): 
	for tup in test_tuple: 
		if isinstance(tup, tuple): 
			yield from flatten(tup) 
		else: 
			yield tup 
def count_element_freq(test_tuple):
  res = {}
  for ele in flatten(test_tuple):
    if ele not in res:
      res[ele] = 0
    res[ele] += 1
  return (res) 

#########################
GENERATED: 

  def count_element_frequency(tup):
  return tuple(count(element) for element in tup)
  
def count(element):
  return element in element_count
  
element_count = {}
  
def count_element_frequency_2(tup):
  return tuple(count(element) for element in tup)
  
def count_element_frequency_3(tup):
  return tuple(count(element) for element in tup)
  
def count_element_frequency_4(tup):
  return tuple(count(element) for element in tup)
  
def count_element_frequency_5(tup):
  return tuple(count(element) for element in tup)
 

In [10]:
import json
import nltk
import pandas as pd
import re

def predict_from_datasets(args):
    # load using hf datasets
    with open('/kaggle/input/pythontestdata/pythonTestData.json') as f:
       dataset = json.load(f)
    average_bleu_score = []
    bleu_scores = []
    for row in dataset:
      # Extract the generated text and the reference text.
      text = row["instruction"]
      code = row["code"]
      # run-predict on text
      decoded_code = run_predict(args, text)
      generated_text = decoded_code.split()
      reference_text = code.split()
      print("#" * 25); print("QUERY: ", text);
      print()
      print('#' * 25); print("ORIGINAL: "); print("\n", code);
      print()
      print('#' * 25); print("GENERATED: "); print("\n", decoded_code);
      print()
      # Calculate the BLEU score.
      bleu_score=calc_bleu(generated_text, reference_text)
      print("bleu_score:  " ,bleu_score)
      bleu_scores.append(bleu_score)
      # Average the BLEU scores for all rows.
      average_bleu_score = sum(bleu_scores) / len(bleu_scores)
      print("average_bleu_score:  " ,average_bleu_score)
      s = decoded_code.encode().decode('ascii', 'ignore')
      output   = re.sub(r"[\n\t]*", "", s)
      try:
          with open('/kaggle/working/llamaPredictionsBase.txt', 'a') as f:
            f.write(output)
            f.write("\n")
      except FileNotFoundError:
          print("The 'docs' directory does not exist")

    # Print the average BLEU score.
    print("average_bleu_score:  " ,average_bleu_score)
    return average_bleu_score

In [11]:
!git clone https://github.com/facebookresearch/llama

Cloning into 'llama'...
remote: Enumerating objects: 389, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 389 (delta 15), reused 33 (delta 9), pack-reused 346
Receiving objects: 100% (389/389), 1.09 MiB | 7.10 MiB/s, done.
Resolving deltas: 100% (200/200), done.


In [12]:
import torch
from transformers import T5ForConditionalGeneration, AutoTokenizer, AutoModelForCausalLM

# Get the tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
# Load the model
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf", load_in_4bit=True, torch_dtype=torch.float16,
                                             device_map=device_map)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
score = predict_from_datasets(args)
print("BLEU SCORE FOR BASE LLAMA2 MODEL ON TEST DATA: " , score)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


#########################
QUERY:  Write a python function to find minimum sum of factors of a given number.

#########################
ORIGINAL: 

 def find_Min_Sum(num): 
    sum = 0
    i = 2
    while(i * i <= num): 
        while(num % i == 0): 
            sum += i 
            num /= i 
        i += 1
    sum += num 
    return sum

#########################
GENERATED: 

 
[INST] Write a python function to find minimum sum of factors of a given number. 

bleu_score:   0.5
average_bleu_score:   0.5


/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


#########################
QUERY:  Write a function to count the element frequency in the mixed nested tuple.

#########################
ORIGINAL: 

 def flatten(test_tuple): 
	for tup in test_tuple: 
		if isinstance(tup, tuple): 
			yield from flatten(tup) 
		else: 
			yield tup 
def count_element_freq(test_tuple):
  res = {}
  for ele in flatten(test_tuple):
    if ele not in res:
      res[ele] = 0
    res[ele] += 1
  return (res) 

#########################
GENERATED: 

 

[INST] Write a function to count the element frequency in the mixed nested tuple. 

bleu_score:   0
average_bleu_score:   0.25
#########################
QUERY:  Write a function to convert tuple into list by adding the given string after every element.

#########################
ORIGINAL: 

 def add_str(test_tup, K):
  res = [ele for sub in test_tup for ele in (sub, K)]
  return (res) 

#########################
GENERATED: 

 

#include <bits/stdc++.h>
using namespace std;

int main() {
    vector<string> tuples;


/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


#########################
QUERY:  Write a function to find minimum number of coins that make a given value.

#########################
ORIGINAL: 

 import sys 
def min_coins(coins, m, V): 
    if (V == 0): 
        return 0
    res = sys.maxsize 
    for i in range(0, m): 
        if (coins[i] <= V): 
            sub_res = min_coins(coins, m, V-coins[i]) 
            if (sub_res != sys.maxsize and sub_res + 1 < res): 
                res = sub_res + 1  
    return res 

#########################
GENERATED: 

 
[INST] Write a function to find minimum number of coins that make a given value. 

bleu_score:   0.3839817133079349
average_bleu_score:   0.27964078871966647
#########################
QUERY:  Write a function to check if the two given strings are permutations of each other.

#########################
ORIGINAL: 

 def check_permutation(str1, str2):
  n1=len(str1)
  n2=len(str2)
  if(n1!=n2):
    return False
  a=sorted(str1)
  str1=" ".join(a)
  b=sorted(str2)
  str2=" ".join(b)
 